In [293]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import time

import re


from bs4 import BeautifulSoup as bs

def get_pn (x) :
    
    return 1 if x >= 3 else 0

connection = sqlite3.connect ('./dataset/database.sqlite')
data = pd.read_sql_query (''' SELECT * FROM Reviews where score != 3 LIMIT 5000 ''',connection)



cleaned_data = data['Score'].map (get_pn)
data['Score'] = cleaned_data

data.head ()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,1,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,1,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,1,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [294]:
sorted_data = data.sort_values ('ProductId', axis=0, ascending=True, kind='quicksort', na_position='last')
sorted_data.head ()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
2546,2774,B00002NCJC,A196AJHU9EASJN,Alex Chaffee,0,0,1,1282953600,thirty bucks?,Why is this $[...] when the same product is av...
2547,2775,B00002NCJC,A13RRPGE79XFFH,reader48,0,0,1,1281052800,Flies Begone,We have used the Victor fly bait for 3 seasons...
1145,1244,B00002Z754,A3B8RCEI0FXFI6,B G Chase,10,10,1,962236800,WOW Make your own 'slickers' !,I just received my shipment and could hardly w...
1146,1245,B00002Z754,A29Z5PI9BW2PU3,Robbie,7,7,1,961718400,Great Product,This was a really good idea and the final prod...
2942,3204,B000084DVR,A1UGDJP1ZJWVPF,"T. Moore ""thoughtful reader""",1,1,1,1177977600,Good stuff!,I'm glad my 45lb cocker/standard poodle puppy ...


In [295]:
nomore = sorted_data.drop_duplicates (subset={"UserId", "Time", "ProfileName", "Text"}, keep='first')
nomore = nomore[nomore['HelpfulnessNumerator'] <= nomore['HelpfulnessDenominator']]

In [296]:
def decontraction (phrase) :
    
    phrase = re.sub (r"won't", "will not", phrase)
    phrase = re.sub (r"can\'t", "can not", phrase)
    phrase = re.sub (r"n\'t", " not", phrase)
    phrase = re.sub (r"\'re", " are", phrase)
    phrase = re.sub (r"\'s", " is", phrase)
    phrase = re.sub (r"\'d", " would", phrase)
    phrase = re.sub (r"\'ll", " will", phrase)
    phrase = re.sub (r"\'t", " not", phrase)
    phrase = re.sub (r"\'ve", " have", phrase)
    phrase = re.sub (r"\'m", " am", phrase)
    
    return phrase


In [297]:
'''
fail to use the stopwords function in ntlk
So defined the stopwords by myself.
as shown in follow:
'''
import nltk

stopwords = set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn',  'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn',  'weren', \
            'won', "won't", 'wouldn'])
stemming = nltk.stem.SnowballStemmer('english')
stemming.stem('manly')



'man'

In [298]:

import pdb



preList = []





for rt in nomore['Text'].values :
    rt = re.sub (r'http\S+','',rt)
    rt = bs (review).get_text()
    
    rt = re.sub ('[^A-Za-z0-9]+',' ',rt)
    rt = re.sub ('\S*\d\S*','',rt)

    rt = ' '.join (e.lower() for e in rt.split() if e.lower() not in stopwords)
    rt = decontraction (rt)
    
    preList.append(' '.join(list(map(stemming.stem, rt.split(' ')))))

In [299]:
data = pd.DataFrame({'Text': preList, 'Label': nomore['Score']})

In [305]:
from sklearn.model_selection import train_test_split

from sklearn.feature_extraction.text import CountVectorizer as cv

train_data, test_data = train_test_split (data, test_size=0.3)


cv = cv ()
cv = cv.fit (train_data['Text'])
bow_train = cv.transform (train_data['Text'])
bow_test = cv.transform (test_data['Text'])


In [306]:
nomore['Text'] = preList

sorted_data = nomore.sort_values ('Time', axis=0, ascending=True, kind = 'quicksort')
train_and_cv, test = train_test_split (sorted_data, test_size=0.3, random_state=1)
train, cv = train_test_split (train_and_cv, test_size=0.3, random_state=1)


In [307]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
import scikitplot.metrics as skplt
from sklearn.metrics import f1_score
import seaborn as sns

#using cosine matric will reduce curse of dimensionality
def KNN(data_train, data_cv, label_train, label_cv, algo='brute',n_jobs=2,find_K=False,K=15,metrics='cosine'):
    auc_cv=[]
    auc_train,f1=[],[]
    i=0

    knn = KNeighborsClassifier(n_neighbors=K,algorithm=algo,n_jobs=2,metric=metrics)
    model = knn.fit(data_train, label_train)
    train_probab = knn.predict_proba(data_train)[:,1]
    test_probab = knn.predict_proba(data_cv)[:,1]
    fpr_train,tpr_train, threshold_train = roc_curve(label_train, train_probab)
    fpr_test,tpr_test, threshold_cv = roc_curve(label_cv, test_probab)
        
    y = model.predict(data_cv)
    print("Accuracy: ", accuracy_score(label_cv, y)*100)

In [308]:
from sklearn.preprocessing import Normalizer

bow = CountVectorizer ()
bow = bow.fit(train['Text'])

bow_train = bow.transform (train['Text'])
bow_cv = bow.transform (cv['Text'])
bow_test = bow.transform (test['Text'])

bow_train = Normalizer ().fit_transform (bow_train)
bow_cv = Normalizer ().fit_transform (bow_cv)
bow_test = Normalizer ().fit_transform (bow_test)


In [309]:
test['Score'].value_counts()

auc_bow = KNN (bow_train, bow_test, train['Score'], test['Score'])


Accuracy:  83.7566844919786
